In [2]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import bisect, brentq
from copy import copy

# Problem 1

## A model for the dynamics
A model of the underlying dynamics captures what we assume about the underlying risks. Previously, we modeled the dynamics of an underlying by geometric Brownian motion (GBM). However, the GBM model is not consistent with the observed non-constant implied volatility skew. An approach to reconcile the matter is a **local volatility model**, where the instantaneous volatility $\sigma(S_t, t)$ is a function of underlying price $S_t$ and time $t$,
$$\mathrm{d}S_t=(r-q)S_t\mathrm{d}t+\sigma(S_t,t)S_t\mathrm{d}W_t$$
- $\sigma=0.4$, volatility
- $S_0=100$, time-0 underlying price 
- $r=0.06$, USD interest rate
- $q=0.01$, GDC interest rate
- $W_t$, Brownian motion under the USD risk-neutral probability measure

The local volatility is
$$\sigma(S,t):=\min[0.2+5(\log(S/100))^2+0.1e^{-t}, 0.6]$$

### Calculate log-underlying dynamics
Start from 
$$\mathrm{d}S_t=(r-q)S_t\mathrm{d}t+\sigma(S_t,t)S_t\mathrm{d}W_t$$
- $R_t=(r-q)S_t$
- $A_t=\sigma(S_t,t)S_t$

Log-transform
$$F(t,S_t)=\log  (S_t)$$
- $\partial _t F(t,S_t)=0$
- $\partial_{s}F(t,S_t)=\frac{1}{S_t}$
- $\partial_{ss} F(t,S_t)=-\frac{1}{S_t^2}$

Ito’s formula - 
$$\begin{aligned}
dF(t, X_t) &= \left[ \partial_tF(t,X_t) + R_t \partial_xF(t,X_t) + \frac{A_t^2}{2} \partial_{xx} F(t,X_t) \right]dt + A_t \partial_xF(t,X_t) dW_t \\
&= \left[(r-q)S_t\cdot \frac{1}{S_t} + \frac{\left(\sigma(S_t,t)S_t\right)^2}{2} \left(-\frac{1}{S_t^2} \right) \right]dt + \sigma(S_t,t)S_t \cdot \frac{1}{S_t} dW_t \\
&= \left[(r-q) - \frac{\sigma^2(S_t,t)}{2} \right]dt + \sigma(S_t,t)dW_t
\end{aligned}$$

In other words,
Logarithm of underlying $F=\log S$ follows dynamics of
$$\mathrm dF_t=\nu\mathrm dt+\sigma\mathrm dW_t$$
- $\nu:=(r-q) - \sigma^2(S_t,t)/2$


## A pricer / solution method
A pricer captures how we price the contract. In a trinomial tree model, the up, middle, down probabilities $p_u, p_m, p_d$ are -
\begin{aligned}
&p_{u} =\frac{1}{2}\bigg[\frac{\sigma^{2}\Delta t+\nu^{2}(\Delta t)^{2}}{(\Delta x)^{2}}+\frac{\nu\Delta t}{\Delta x}\bigg] \\
&p_{m} =1-\frac{\sigma^{2}\Delta t+\nu^{2}(\Delta t)^{2}}{(\Delta x)^{2}} \\
&p_{d} =\frac{1}{2}\bigg[\frac{\sigma^{2}\Delta t+\nu^{2}(\Delta t)^{2}}{(\Delta x)^{2}}-\frac{\nu\Delta t}{\Delta x}\bigg] 
\end{aligned}

For consistency and stability reasons, we pick the time step and space step parameters to be - 
- $\Delta t:=T/N$, time step
- $\Delta x=\sigma \sqrt{3\Delta t}$, up/down move size, picked for accuracy reasons



## Contracts to be priced
- `1a`, American-style put
- `1b`, a compound option - a European-style call on an American-style put

In [52]:
class localvolDynamics:

    def __init__(self, S0, r, q, maxvol, localvol):
        self.S0 = S0
        self.r = r
        self.q = q
        self.maxvol = maxvol
        self.localvol = localvol


In [53]:
hw2dynamics = localvolDynamics(S0 = 100, r = 0.06, q = 0.01, maxvol = 0.6,
                     localvol = lambda S,t: np.minimum(0.2+5*np.log(S/100)**2+0.1*np.exp(-t), 0.6))

# Note that hw2dynamics.localvol is a function
# that may be invoked in the usual way, for example:
# hw2dynamics.localvol( exchangerate , time )

In [54]:
class CallOnAmericanPut:

    def __init__(self, putexpiry, putstrike, callexpiry, callstrike):
        self.putexpiry = putexpiry
        self.putstrike = putstrike
        self.callexpiry = callexpiry
        self.callstrike = callstrike


In [55]:
hw2contract = CallOnAmericanPut(putexpiry=0.75, putstrike=95, callexpiry=0.25, callstrike=10)

In [77]:
class TreeEngine:

    def __init__(self, N):
        self.N = N

    # You complete the coding of this function
    def price_compound_localvol(self, contract, dynamics):

        # Set up parameters: time step, space step
        deltat = contract.putexpiry / self.N
        # deltax chosen as a tradeoff between stability and local discretization error
        # localvol_avg estimated by first point of localvol
        deltax = np.maximum(dynamics.maxvol * np.sqrt(deltat),
                        dynamics.localvol(dynamics.S0, 0) * np.sqrt(3 * deltat))
        
        # Set up grid
        Sgrid = dynamics.S0 * np.exp(np.linspace(
                    self.N, -self.N, num=2*self.N+1, endpoint=True) * deltax)



        # Initialize option values at expiry
        put_prices = np.maximum(contract.putstrike - Sgrid, 0)

         # Initialize call on put option values at expiry
        flag = True
        # Roll back the option prices
        for i in range(self.N-1, -1, -1):
            t = i * deltat
            Sgrid = Sgrid[1:-1]

            # Update trinomial probabilities
            nu = dynamics.r - dynamics.q - 0.5 * dynamics.localvol(Sgrid, t) ** 2
            Pu = 0.5 * ((dynamics.localvol(Sgrid, t) ** 2 * deltat 
                         + nu ** 2 * deltat ** 2) / (deltax ** 2) 
                        + nu * deltat / deltax)
            Pd = 0.5 * ((dynamics.localvol(Sgrid, t) ** 2 * deltat 
                         + nu ** 2 * deltat ** 2) / (deltax ** 2) 
                        - nu * deltat / deltax)
            Pm = 1 - Pu - Pd

            # Interpolate to get the American-style put price
            put_prices = np.exp(-dynamics.r * deltat) * (Pu * put_prices[:-2] + Pm * put_prices[1:-1] + Pd * put_prices[2:])
            # Check for early exercise, since American-style option
            put_prices = np.maximum(put_prices, np.maximum(contract.putstrike - Sgrid, 0))

            # Interpolate to get the European-style call on put price
            if t < contract.callexpiry + deltat:
                if flag:
                    # Initialize once
                    call_on_put_prices = np.maximum(put_prices - contract.callstrike, 0)
                    flag = False
                    continue
                call_on_put_prices = np.exp(-dynamics.r * deltat) * (Pu * call_on_put_prices[:-2] + Pm * call_on_put_prices[1:-1] + Pd * call_on_put_prices[2:])

        price_of_put = put_prices[0]

        price_of_call_on_put = call_on_put_prices[0]     #write code to compute this

        return (price_of_put, price_of_call_on_put)


In [80]:
hw2tree = TreeEngine(N=1000)  #change if necessary to get $0.01 accuracy, in your judgment

In [81]:
(answer_part_a, answer_part_b) = hw2tree.price_compound_localvol(hw2contract,hw2dynamics)

In [82]:
(answer_part_a, answer_part_b)

(7.004324880881289, 1.5954402511696555)

# Problem 2

### `2a`: first-order Taylor expansion of $\Delta$

Black-Scholes Vanilla Call pricing:
$$f(S):=(S-K)^{+}$$
$$C(S,t)=C^{BS}(S,t,K,T,r-q,r,\sigma)=S_0N(d_1)-Ke^{-rT}N(d_2)$$
where
$$d_{1,2}:=d_{+,-}:=\frac{\log(S_0e^{rT}/K)}{\sigma\sqrt T}\pm\frac{\sigma\sqrt T}2$$
In our case $t=0$, $r=0$.

Combined, 
$$C(S,0)=S_0 N\left(\frac{\log(S_0e^{rT}/K)}{\sigma\sqrt T} + \frac{\sigma\sqrt T}2\right)
-Ke^{-rT}N\left(\frac{\log(S_0e^{rT}/K)}{\sigma\sqrt T} - \frac{\sigma\sqrt T}2\right)$$

Approximate delta by applying a first-order Taylor expansion to the exact formula, as follows - 
$$\Delta:=\frac{\partial C}{\partial S}\approx\frac{C(S+\Delta S,t)-C(S,t)}{\Delta S}$$

For ignoring the term inside the normal distribution for simplicity, for at-the-money call,
$$\Delta=N(d_1) \approx N\left(\frac{rT}{\sigma\sqrt T}+ \frac{\sigma\sqrt T}2\right)$$

In [14]:
sigma = 0.2
T = 0.25
r = 0.01

# at-the-money call option
d1 = r * T / (sigma * np.sqrt(T)) + 0.5 * sigma * np.sqrt(T)
delta = norm.cdf(d1)
print(f"ATM Call Option Delta = {delta: .2f}")

print(delta)

0.5298926440528947


### `2b`: second-order Taylor expansion of an option

Delta - the shares of stock sold per dollar movement in $S$,
$$\Delta=\frac{\partial C}{\partial S}$$
Time-o dollar delta - the dollars of stock sold per dollar movement in $S$, 
$$\tilde{\Delta}=S_0\frac{\partial C}{\partial S}$$
Normal gamma - the shares of stock sold per dollar movement in $S$, 
$$\Gamma =\frac{\partial^2 C}{\partial S^2}$$
Time-o dollar gamma - the dollars of stock sold per $1\%$ change in $S$, 
$$\tilde{\Gamma }=\frac{S_0^2}{100} \times \frac{\partial^2 C}{\partial S^2}$$

Taylor expansion to second-order, 
$$C(S)\approx C(S_0)+(S-S_0)\cdot \frac{\partial C}{\partial S}+ \frac12 (S-S_0)^2\frac{\partial^2C}{\partial S^2} $$
$$ \approx C(S_0) + (S - S_0) \cdot \Delta + \frac{(S - S_0)^2}{2} \cdot \Gamma $$

In [17]:
S0 = 4
C_S0 = 5
delta_dollar = 3
gamma_dollar = 0.02

S = 3.6

delta = delta_dollar / S0
gamma = gamma_dollar * 100 / (S0 ** 2)

price = C_S0 + delta * (S - S0) + 0.5 * gamma * (S - S0) ** 2
print(f"The time-0 value of the contract = {price: .2f}")

The time-0 value of the contract =  4.71
